#### Import Necessari

In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier


#### Caricamento Dataset

In [2]:
train1 = pd.read_json('dataset/Dataset20-21.json');
train2 = pd.read_json('dataset/Dataset21-22.json');
trainSet = pd.concat([train1, train2], ignore_index=True)
validationSet = pd.read_json('dataset/Dataset22-23.json');
testSet = pd.read_json('dataset/Dataset19-20.json');
trainSet.loc[trainSet["risultato"] == "X", "risultato"] = 3
trainSet.loc[trainSet["risultato"] == "x", "risultato"] = 3
validationSet.loc[validationSet["risultato"] == "x", "risultato"] = 3
validationSet.loc[validationSet["risultato"] == "X", "risultato"] = 3

In [3]:
X_train = trainSet.drop("risultato", axis=1)
y_train = trainSet["risultato"]

X_validation = validationSet.drop("risultato", axis=1)
y_validation = validationSet["risultato"]

X_test = testSet.drop("risultato", axis=1)
y_test = testSet["risultato"]


In [4]:
X_train_noName = X_train
X_train_noName = X_train_noName.drop("squadraa", axis=1)
X_train_noName = X_train_noName.drop("squadrab", axis=1)
X_train_noName = X_train_noName.astype(float)
y_train=y_train.astype(float)

X_validation_noName = X_validation
X_validation_noName = X_validation_noName.drop("squadraa", axis=1)
X_validation_noName = X_validation_noName.drop("squadrab", axis=1)
X_validation_noName = X_validation_noName.astype(float)
y_validation=y_validation.astype(float)

X_test_noName = X_test
X_test_noName = X_test_noName.drop("squadraa", axis=1)
X_test_noName = X_test_noName.drop("squadrab", axis=1)
X_test_noName = X_test_noName.astype(float)
y_test=y_test.astype(float)




#### Inizio Codice training

In [5]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import precision_score, recall_score

selector = SelectKBest(f_regression, k=5)
X_train_noName = selector.fit_transform(X_train_noName, y_train)
X_validation_noName = selector.transform(X_validation_noName)
X_test_noName = selector.transform(X_test_noName)

def evaluate_model(n_estimators, max_depth, X_train, y_train, X_validation, y_validation):
    rf_model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_validation)
    accuracy = accuracy_score(y_validation, y_pred)
    return accuracy

best_n_estimators = 100
best_max_depth = None  # Non specificare una profondità massima iniziale
best_accuracy = 0.0
max_iterations = 50

mask = ~np.isnan(y_validation)
y_validation = y_validation[mask]
X_validation_noName = X_validation_noName[mask] # Rimozione righe con NaN Values


for _ in range(max_iterations):
    # Genera nuovi iperparametri casuali
    new_n_estimators = np.random.randint(50, 201)
    new_max_depth = np.random.choice([None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

    # Valuta il modello con i nuovi iperparametri
    accuracy = evaluate_model(new_n_estimators, new_max_depth, X_train_noName, y_train, X_validation_noName, y_validation)

    # Verifica se abbiamo trovato iperparametri migliori
    if accuracy > best_accuracy:
        best_n_estimators = new_n_estimators
        best_max_depth = new_max_depth
        best_accuracy = accuracy

final_model = RandomForestClassifier(n_estimators=best_n_estimators, max_depth=best_max_depth, random_state=42)
final_model.fit(X_train_noName, y_train)
y_pred_validation = final_model.predict(X_validation_noName)
y_test_pred = final_model.predict(X_test_noName)
accuracy_test = accuracy_score(y_test, y_test_pred)
finalPrecision = precision_score(y_test, y_test_pred, average='weighted', zero_division=1)
finalRecall = recall_score(y_test, y_test_pred, average='weighted', zero_division=1)
print("Best Hyperparameters: n_estimators =", best_n_estimators, ", max_depth =", best_max_depth)
print(f"Accuracy on Test Data: {accuracy_test}")
print(f"Precision: {finalPrecision}")
print(f"Recall: {finalRecall}")
